In [ ]:
import segmentation_models as sm
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import cv2
import matplotlib.pyplot as plt

BACKBONE = 'vgg16'
model=sm.Unet(BACKBONE,input_shape=(224,224,1),encoder_weights=None,weights='D:/ML_project/segmentation_models-master/segmentation_models-master/train_summary/vgg16/bestvgg16.h5')

In [ ]:

dirpath = 'D:/ML_project/test/images_new_selected/'

imgpths=[]

for pth in os.listdir(dirpath):
    if '.png' in pth:
        imgpths.append(os.path.join(dirpath,pth))


In [ ]:
val_mask = 'D:/ML_project/test/masks_new_selected/'


In [ ]:
roc_points=[]
th_list=np.arange(0,260,5)
for th in th_list:
#     print("th=",th)
    tp_l = []
    tn_l = []
    fp_l = []
    fn_l = []
    for ipth in imgpths[:]:
        basename=os.path.basename(ipth)
        val_gt=os.path.join(val_mask,basename)
        g_t=cv2.imread(val_gt,0)
        img=cv2.imread(ipth,0)
        rimg=img.copy()
        g_t_img=img.copy()
        y,x=img.shape[:2]
        img=cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        img=img/255

        img = np.reshape(img,(1,)+img.shape)
        img = np.reshape(img,img.shape+(1,))
        # print(img.shape)
        seg_res=model.predict(img)[0]
        seg_res=np.array(seg_res*255)
        seg_res=cv2.convertScaleAbs(seg_res)
        seg_res = cv2.resize(seg_res, (x, y), interpolation=cv2.INTER_CUBIC)
        seg_res_copy=seg_res.copy()

        
        ret, seg_res = cv2.threshold(seg_res_copy, th, 255, cv2.THRESH_BINARY)
        g_t[g_t == 255] = 1
        seg_res[seg_res == 255] = 1
        tp = (g_t * seg_res).sum()
        tn = ((1 - g_t) * (1 - seg_res)).sum()
        fp = ((1 - g_t) * seg_res).sum()
        fn = (g_t * (1 - seg_res)).sum()

        tp_l.append(tp)
        tn_l.append(tn)
        fp_l.append(fp)
        fn_l.append(fn)
        

    tpr = sum(tp_l) / (sum(tp_l) + sum(fn_l))
    fpr = sum(fp_l) / (sum(tn_l) + sum(fp_l))
    roc_points.append([fpr,tpr])



In [ ]:
fpr_array = []
tpr_array = []
for i in range(len(roc_points) - 1):
    point1 = roc_points[i]
    point2 = roc_points[i + 1]
    tpr_array.append([point1[0], point2[0]])
    fpr_array.append([point1[1], point2[1]])

auc = sum(np.trapz(tpr_array,fpr_array))+1
print('Area under curve={}'.format(auc))

plt.plot(tpr_array,fpr_array, 'r', lw=2)
plt.plot([0, 1], [0, 1], 'k-', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC={}'.format(round(auc,3)))
plt.show()



In [ ]:
x=[]
y=[]
for i in range(len(roc_points)):
    points1 = roc_points[i]
    x.append(points1[0])
    y.append(points1[1])

plt.plot(x,y, 'r', lw=2)
plt.plot([0, 1], [0, 1], 'k-', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC={}'.format(round(auc,3)))
plt.show()

In [ ]:
len(roc_points)

In [ ]:
Finding IOU

In [ ]:
iou_points=[]
th_list=np.arange(0,260,5)
for th in th_list:
    print("th=",th)
    iou_list=[]
    for ipth in imgpths[:]:
        basename=os.path.basename(ipth)
        val_gt=os.path.join(val_mask,basename)
        g_t=cv2.imread(val_gt,0)
        img=cv2.imread(ipth,0)
        rimg=img.copy()
        g_t_img=img.copy()
        y,x=img.shape[:2]
        img=cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
        img=img/255

        img = np.reshape(img,(1,)+img.shape)
        img = np.reshape(img,img.shape+(1,))
        # print(img.shape)
        seg_res=model.predict(img)[0]
        seg_res=np.array(seg_res*255)
        seg_res=cv2.convertScaleAbs(seg_res)
        seg_res = cv2.resize(seg_res, (x, y), interpolation=cv2.INTER_CUBIC)
        seg_res_copy=seg_res.copy()

        
        ret, seg_res = cv2.threshold(seg_res_copy, th, 255, cv2.THRESH_BINARY)
        def iou_score(g_t,seg_res):
            mask1_area = np.count_nonzero(g_t == 255)
            mask2_area = np.count_nonzero(seg_res == 255)
            intersection = np.count_nonzero( np.logical_and( g_t, seg_res) )
            iou = round(((intersection/(mask1_area+mask2_area-intersection))*100),2)
            return iou
        iou_list.append(iou_score(g_t,seg_res))


    iou_points.append(np.mean(iou_list))


In [ ]:
max(iou_points)

In [ ]:
index=np.argmax(np.array(iou_points))

In [ ]:
max_iou=iou_points[index]
req_th=th_list[index]

In [ ]:
%matplotlib qt
plt.plot(th_list,iou_points, 'r', lw=2)
plt.plot(req_th,max_iou,'*',markersize=20)
ll="Selected threshold="+str(req_th)+" , "+"Max IOU="+str(max_iou)
plt.title(ll)
plt.ylabel('IOU')
plt.xlabel('Thresholds')
plt.show()

In [4]:
from skimage import exposure
import cv2
import matplotlib.pyplot as plt
import os

In [5]:
%matplotlib qt

In [10]:

dirpath = 'D:/ML_project/test/images_new_selected/'

imgpths=[]

print(len(os.listdir(dirpath)))

        

302


In [13]:
print("hi")
for pth in os.listdir(dirpath):
#     print("h")
#     if '.png' in pth[10:]:
#         print("hi")
        img=cv2.imread(os.path.join(dirpath,pth),0)
        
        gamma_corrected = exposure.adjust_gamma(img, 0.5)
        plt.subplot(121),plt.imshow(img,'gray')
        plt.subplot(122),plt.imshow(gamma_corrected,'gray')
        plt.show()

hi


C:\Users\sgpra\anaconda3\envs\ml_low_tf_bk\lib\site-packages\ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if __name__ == '__main__':
C:\Users\sgpra\anaconda3\envs\ml_low_tf_bk\lib\site-packages\ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  # Remove the CWD from sys.path while we load stuff.
